# 1회차 수업

# 1. 데이터 분석과 전처리: 왜 이게 먼저인가?

In [265]:
import pandas as pd

## 1.1) 데이터 분석 전체 흐름의 정확한 이해
데이터 분석은 보통 다음 순서로 진행된다.
1. 데이터 수집(파일/DB/API)
2. 전처리(정제, 형변환, 결측치, 이상치, 파생변수)
3. 탐색(EDA: 분포, 관계, 패턴, 가설)
4. 시각화(의사소통 + 패턴 확인)
5. 모델링(예측/분류/추천 등)
6. 결과 해석/의사결정

이 순서는 권장 사항이 아니라 필수 순서다.
- 전처리가 끝나지 않으면 그 이후 단계는 정상적으로 작동하지 않기 때문이다

## 1.2) 전처리는 "준비 단계"가 아닌 "전제 조건"이다
전처리는 데이터가 계산·비교·집계·시각화가 가능하도록 구조와 타입을 맞추는 작업이다.

## 1.3) 전처리를 안 하면 발생하는 문제
1. 계산이 불가능해진다
    - 숫자로 계산해야 할 값이 문자열이면 곱셈, 합계, 평균이 아예 동작하지 않는다
2. 결과의 외곡
    - 같은 의미의 값이 다른 값으로 인식되면, 집계 결과가 쪼개진다
3. 시각화/모델 오류
    - 그래프가 안 그려지거나 모델 학습 단계에서 오류가 난다

기억하자. 데이터 분석 시간의 절반 이상이 전처리다

## 1.4) 피처(Feature)를 정확히 이해해야 전처리가 보인다
피처란 분석에 실제로 사용하는 컬럼이다. (단, 모든 컬럼이 피처는 아니다.)

### 분석 목표가 바뀌면 피처도 바뀐다
같은 데이터라도 목적이 다르면 필요한 컬럼이 달라진다.
- 메뉴별 매출이면: menu, price, qty, date
- 결제 성공률이면: paid, date, menu

기억하자. 전처리는 "모든 컬럼을 깨끗하게 만드는 작업"이 아닌 "사용할 피처를 분석 가능하게 만드는 작업"이다.

## 1.5) 망가진 원본 데이터를 살펴보기

In [266]:
# -----------------------------------------
# 일부러 문제가 섞인 원본 데이터
# -----------------------------------------
raw = [
    {"date": "2026-01-01", "menu": "Americano", "price": "4500원", "qty": "2", "paid": "TRUE"},
    {"date": "2026/01/01", "menu": "Latte",     "price": "5,000",  "qty": 1,   "paid": "True"},
    {"date": "2026-01-02", "menu": "Latte",     "price": None,     "qty": 2,   "paid": "FALSE"},
    {"date": "2026-01-03", "menu": "Mocha",     "price": "5500",   "qty": None,"paid": True},
]
df = pd.DataFrame(raw)

### 이 데이터의 문제는 무엇일까?
- price: 문자열 + 기호
- qty: 타입 섞임
- 결측치 존재
- date 포맷 불일치
- paid 표기 불일치

이 상태에서는 매출 계산이 절대적으로 불가능하고 해서는 안된다.

## 1.6) 전처리의 사고 순서 (이게 핵심)
전처리를 할때 항상 이 질문과 함께 시작하자
- 이 컬럼은 분석에서 어떤 역할을 해야 하는가?

### 한번 해보자. price 전처리: 문자열을 숫자로 만드는 이유
1. price는 매출 계산에 쓰이기 때문에 숫자형이여야 한다.
2. 현재 데이터는 문자열 + 기호가 섞여 있다
3. 불필요한 문자를 제거하고 숫자로 바꾼다

In [267]:
df["price_num"] = (
    df["price"]
    .astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
# price 컬럼을 문자열로 통일한 뒤, 숫자 계산에 방해되는 기호를 제거한다

df["price_num"] = pd.to_numeric(df["price_num"], errors="coerce")
# price_를 다시 숫자형으로 변환, 결측치로 처리한다.

### 한번 해보자. qty 전처리: 타입을 하나로 통일한다
1. 수량은 개수이므로 반드시 숫자 타입이어야 한다

In [268]:
df["qty_num"] = pd.to_numeric(df["qty"], errors="coerce")
# 변환보단 타입 통일 작업으로 인식

### 한번 해보자. 파생 피처(derived feature)란?
sales는 원본 데이터에는 없었다 하지만 분석에서는 가장 중요한 값이다. <br>
때문에 컬럼을 직접 만들어야 한다. 이러한것을 파생 피처라 부른다.<br>
즉, 파생 피처란 원본에는 없지만 분석 목적을 위해 새로 만든 컬럼이다.

In [269]:
df["sales"] = df["price_num"] * df["qty_num"]

# 2) Pandas가 다루는 데이터 구조: Series와 DataFrame

## 2.1) Pandas에는 데이터 구조가 딱 두 개뿐이다
Pandas는 복잡해 보이지만 핵심 데이터 구조는 두 개뿐이다.
- Series
- DataFrame

Pandas에서 하는 모든 연산은 결국 Series 또는 DataFrame을 대상으로 한다.

## 2.2) DataFrame부터 정확히 정의하자
DataFrame은 여러 개의 Series가 같은 index를 공유하며 모여 있는 구조다.

- 표라는 말은 결과 모습일 뿐
- 내부 구조는 Series들의 묶음 이다

### DataFrame의 구성요소
1. info(): 컬럼 수, dtype, 결측치, 행 수를 한 번에 파악한다.
2. shape: 데이터 규모(행, 열) 확인한다.
3. columns: 컬럼 이름 목록만 빠르게 확인할 때 사용한다.
4. dtypes: 컬럼들의 자료형 파악. (info()에 포함된 정보)
5. index: 행 번호(혹은 의미있는 키)

```python
df.info()
df.shape
df.index
df.columns
df.dtypes
```

## 2.3) Series도 정의하자
Series는 하나의 컬럼을 구성하는 1차원 데이터 구조다.

### Series의 구성요소
1. info(): 컬럼 수, dtype, 결측치, 행 수를 한 번에 파악한다.
2. shape: 데이터 규모(행, 열) 확인한다.
3. columns: 컬럼 이름 목록만 빠르게 확인할 때 사용한다.
4. dtype: 컬럼의 자료형 파악 (info()에 포함된 정보)
5. index: 행 번호(혹은 의미있는 키)

```python
df.info()
df.shape
df.index
df.columns
df.dtypes
```

#### 자주 범하는 오류
dtype는 dtypes와는 다르다! dtypes는 DataFrame의 컬럼의 자료형, dtype이다. 

## 2.4) 코드로 확인해보자

In [228]:
df = pd.DataFrame({
    "date": ["2026-01-01", "2026-01-01", "2026-01-02", "2026-01-03"],
    "menu": ["Americano", "Latte", "Latte", "Mocha"],
    "price": [4500, 5000, None, 5500],
    "qty": [2, 1, 2, None],
    "paid": ["TRUE", "True", "FALSE", True],
})

df

,date,menu,price,qty,paid
0,2026-01-01,Americano,4500.0,2.0,TRUE
1,2026-01-01,Latte,5000.0,1.0,True
2,2026-01-02,Latte,NaN,2.0,FALSE
3,2026-01-03,Mocha,5500.0,NaN,True


In [229]:
print("데이터 프레임의 종합 정보")
print(df.info())

print("\n데이터 프레임의 규모 확인")
df.shape


데이터 프레임의 종합 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4 non-null      object 
 1   menu    4 non-null      object 
 2   price   3 non-null      float64
 3   qty     3 non-null      float64
 4   paid    4 non-null      object 
dtypes: float64(2), object(3)
memory usage: 288.0+ bytes
None

데이터 프레임의 규모 확인


(4, 5)

In [230]:
s = df["paid"]

print("데이터 프레임의 종합 정보")
print(s.info())

print("\n데이터 프레임의 규모 확인")
print(s.shape)


데이터 프레임의 종합 정보
<class 'pandas.core.series.Series'>
RangeIndex: 4 entries, 0 to 3
Series name: paid
Non-Null Count  Dtype 
--------------  ----- 
4 non-null      object
dtypes: object(1)
memory usage: 160.0+ bytes
None

데이터 프레임의 규모 확인
(4,)


## 2.5) Series로 테이블 구성: 대괄호 한 개 vs 두 개 (중요)
기억하기
- Series와 DataFrame은 단순 형태의 차이가 아니다.
- 동작 방식 자체가 다르다

In [231]:
# 컬럼 하나 → Series
print(type(df["paid"]))

# 컬럼 하나짜리 표 → DataFrame
print(type(df[["paid"]]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


## 2.6) 전처리는 왜 Series 단위로 이루어질까?
Pandas의 대부분의 전처리 함수는 Series를 대상으로 설계되어 있다

```Python
df["price"].astype(int)
df["paid"].str.lower()
df["qty"].fillna(0)
```
이러한 작업들은 DataFrame 전체가 아니라 컬럼 하나(Series) 에 적용된다

 # 3) dtype(자료형): 전처리의 핵심 엔진

## 3.1) 왜 dtype가 중요한가?
dtype는 단순 “자료형 정보”가 아니다. <br>
dtype란 Pandas가 해당 컬럼을 어떻게 계산하고, 비교하고, 해석할지를 결정하는 기준이자 규칙이다.


## 민약 틀린다면?
1. 분명 보이는건 숫자인데 계산이 안 된다
2. 날짜인데 날짜 연산이 안 된다
3. True/False인데 필터가 이상하다

## 만약 맞는다면?
전처리에서 중요한 건 "원본 dtype"이 아닌 "분석 가능한 dtype" 이다.
- 가격 → 숫자형이어야 한다
- 날짜 → datetime이어야 한다
- 상태값 → bool 또는 범주형이어야 한다

### 코드로 풀어보자

In [232]:
df = pd.DataFrame({
    "date": ["2026-01-01", "2026/01/02", "2026-01-03"],
    "menu": ["Americano", "Latte", "Mocha"],
    "price": ["4500원", "5,000", None],
    "qty": ["2", 1, None],
    "paid": ["TRUE", "FALSE", True],
})
df

,date,menu,price,qty,paid
0,2026-01-01,Americano,4500원,2,TRUE
1,2026/01/02,Latte,"5,000",1,FALSE
2,2026-01-03,Mocha,None,None,True


In [233]:
print("[1] df.info()")
print(df.info())

print("\n[2] df.shape")
print(df.shape)

print("\n[3] df.dtypes")
print(df.dtypes)

[1] df.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   menu    3 non-null      object
 2   price   2 non-null      object
 3   qty     2 non-null      object
 4   paid    3 non-null      object
dtypes: object(5)
memory usage: 248.0+ bytes
None

[2] df.shape
(3, 5)

[3] df.dtypes
date     object
menu     object
price    object
qty      object
paid     object
dtype: object


겉으로 보면 문제가 없어보이는 테이블도 확인해보면 문제가 많다.
- 숫자처럼 보이는 값들이 문자열이다
- 날짜처럼 보이지만 날짜가 아니다
- True/False가 섞여 있다
즉, 의미는 맞아 보이지만 dtype는 전부 틀려 있다.

### 전처리 목표를 먼저 세우고 시작하자.
전처리는 코드부터 치는 작업이 아니다. 항상 목표 dtype을 문장으로 먼저 정하자. <br>
이번 데이터의 목표는 다음과 같다.

- date → datetime
- price → 숫자
- qty → 숫자
- paid → bool


#### 분석 가능한 dtype로 전처리하기 (의미 중심)

date → datetime

In [234]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
# 날짜로 해석 가능한 값만 datetime으로 바꾸고 불가능한 값은 결측치로 처리한다

price → 숫자

In [235]:
df["price"] = (
    df["price"]
    .astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

# 숫자 계산을 방해하는 문자 제거
# 숫자로 변환 불가능한 값은 결측치 처리

qty → 숫자

In [236]:
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")
# 이건 계산을 위한 타입 통일 작업이다.

paid → bool

In [237]:
df["paid"] = (
    df["paid"]
    .astype("string")
    .str.upper()
    .map({"TRUE": True, "FALSE": False})
)
# 같은 의미의 값을 하나의 논리형 표현으로 통일한다

#### 전처리 후 dtype를 다시 확인해하기
무엇이 바뀌였나?
- date → datetime64
- price, qty → float64 (결측치가 있으면 int가 아니라 float로 보이는 것이 정상)
- paid → bool

In [238]:
df

,date,menu,price,qty,paid
0,2026-01-01,Americano,4500,2.0,True
1,NaT,Latte,5000,1.0,False
2,2026-01-03,Mocha,<NA>,NaN,True


In [239]:
print(df.dtypes)

date     datetime64[ns]
menu             object
price             Int64
qty             float64
paid               bool
dtype: object


### 전처리 메소드 잠깐 정리 (중간 체크)

#### to_numeric
문자열 → 숫자 변환 (전처리에서 가장 많이 씀) <br>
이 컬럼은 숫자처럼 보이지만 실제로는 문자열일 때 사용된다.

기본형태
``` python
pd.to_numeric(컬럼, errors="coerce")
```
숫자로 변환 가능한 값은 숫자로 바꾸고 불가능한 값은 NaN으로 만든다

#### to_datetime
문자열 → 날짜/시간 변환 <br>
날짜처럼 보이는 값으로 월별, 요일별, 기간 분석을 하고 싶을 때 사용된다.

기본형태
``` python
pd.to_datetime(컬럼, errors="coerce")
```
날짜로 해석 가능한 값만 datetime으로 변환한다

#### astype
타입을 바꿔서 다루겠다 는 선언<br>
이미 값이 "변환 가능한 형태"라고 확신할 수 있을 때 사용하는 방식이다.

기본형태
``` python
컬럼.astype("string")
```
이 컬럼을 문자열 타입 기준으로 다루겠다고 Pandas에 알려준다.

#### .str
Series에 문자열 전용 메소드를 쓰겠다는 접근자<br>
문자열 dtype 여부와 상관없이, 각 값에 문자열 연산을 적용할 때 반드시 거쳐야 한다.

#### 가볍게 짚고 가기 (아는 내용)

#### replace
불필요한 문자 제거
- 숫자 계산을 방해하는 문자나 기호를 제거할 때 사용
``` python
.str.replace("원", "")
.str.replace(",", "")
```

#### upper() / lower()
표기 통일용
- 같은 의미인데 대소문자만 다른 값을 통일할 때
``` python
.str.upper()
.str.lower()
```

#### map
의미 → 값 매핑
- 문자열 의미를 실제 분석용 값으로 바꿀 때
- 의미가 같은 값들을 하나의 기준 값으로 치환한다
``` python
.map({"TRUE": True, "FALSE": False})
```


## 3.2) 실무에서 자주 쓰는 dtype 5종 (필수)
앞으로 나오게 될 이 다섯 가지는 분석용 데이터에서 거의 90% 이상을 차지하는 dtype다.<br>
dtype 분류는 Pandas 기준이 아닌 분석에서 어떤 연산을 할 것이냐 기준이다.

In [240]:
df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "price": "4,500원", "qty": "2",  "paid": "TRUE"},
    {"date": "2026/01/02", "menu": "Latte",     "price": "5000",   "qty": 1,    "paid": "False"},
    {"date": None,         "menu": "Mocha",     "price": None,     "qty": None, "paid": True},
    {"date": "invalid",    "menu": "Latte",     "price": "5,000원","qty": "3",  "paid": "TRUE"},
])

print("=== 원본 df ===")
print(df)
print("\n원본 dtypes")
print(df.dtypes)

=== 원본 df ===
         date       menu   price   qty   paid
0  2026-01-01  Americano  4,500원     2   TRUE
1  2026/01/02      Latte    5000     1  False
2        None      Mocha    None  None   True
3     invalid      Latte  5,000원     3   TRUE

원본 dtypes
date     object
menu     object
price    object
qty      object
paid     object
dtype: object


### 1) Numeric (int / float)
숫자 계산이 목적이면 dtype는 반드시 Numeric이어야 한다.

- 합계, 평균, 곱셈, 정렬 같은 수치 계산의 대상이면 Numeric이다.
    - qty
    - price
    - sales

핵심 포인트
- 결측치가 하나라도 섞이면 int가 아니라 float로 보이는 경우가 많다
    - 이건 오류가 아니라 정상 작동

In [241]:
# price는 숫자처럼 보이지만 문자 + 기호가 섞여 있음
# 계산을 위해 불필요한 문자를 제거한 뒤 숫자로 변환
df["price"] = (
    df["price"]
    .astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

# qty 역시 계산 대상이므로 숫자로 통일
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")

# 파생 피처: 계산 결과로 만들어진 Numeric 컬럼
df["sales"] = df["price"] * df["qty"]

print("[Numeric] price dtype:", df["price"].dtype)
print("[Numeric] qty   dtype:", df["qty"].dtype)
print("[Numeric] sales dtype:", df["sales"].dtype)


[Numeric] price dtype: Int64
[Numeric] qty   dtype: float64
[Numeric] sales dtype: Float64


### 2) Text (object / string)
계산 대상이 아니라 이름, 분류, 설명 역할을 하면 Text다.
    - menu

중요한 함정:
- "5,000원"은 숫자처럼 보여도 Text다
- 겉모습이 아니라 연산 목적으로 판단한다.

핵심 포인트
- 계산하지 않는 값은 Text로 시작하는 것이 정상이다.

In [242]:
# menu는 계산 대상이 아니라 '이름/분류' 역할
# 문자열 메소드를 안정적으로 쓰기 위해 명시적으로 string dtype으로 통일
df["menu"] = df["menu"].astype("string")

print("[Text] menu dtype:", df["menu"].dtype)

[Text] menu dtype: string


### 3) Boolean (bool)
- 조건 필터링의 기준이 되면 Boolean이다.
    - paid (결제 여부)

중요한 함정:
- "TRUE", "False" 같은 문자열은 Boolean이 아니다
- 그대로 쓰면 필터 결과가 틀어진다

핵심 포인트
- 조건 판단에 쓰이는 컬럼은 반드시 bool로 통일한다.

In [243]:
# paid는 결제 여부 판단용 컬럼
# 문자열 True/False를 실제 bool 값으로 통일
paid_map = {"TRUE": True, "FALSE": False, "True": True, "False": False}

df["paid"] = (
    df["paid"]
    .astype("string")   # 문자열 기준으로 통일한 뒤
    .map(paid_map)      # 의미를 bool 값으로 매핑
)

print("[Boolean] paid dtype:", df["paid"].dtype)

[Boolean] paid dtype: bool


### 4) Datetime (datetime64)
날짜를 기준으로 그룹핑, 기간 필터, 시계열 분석을 하면 Datetime이다.
    - date

중요한 함정:
- 문자열 상태에서는 날짜 연산이 불가능
- datetime으로 바뀌는 순간 분석이 열린다

핵심 포인트
- 날짜 의미를 가진 컬럼은 반드시 datetime이어야 한다.

In [244]:
# date는 '날짜 의미'를 가지므로 문자열이 아닌 datetime이어야 함
# 날짜로 해석 불가능한 값은 NaT(날짜형 결측)로 처리
df["date"] = pd.to_datetime(df["date"], errors="coerce")

print("[Datetime] date dtype:", df["date"].dtype)

[Datetime] date dtype: datetime64[ns]


### 5) Missing (NaN / NaT)
Missing은 dtype가 아니다. “자료형”이 아니라 값이 없는 상태다.

-숫자형 결측 → NaN
- 날짜형 결측 → NaT

In [245]:
# dtype 변환 실패나 원본 결측은 Missing으로 나타남
# 수치형 결측 → NaN, 날짜형 결측 → NaT
print("[Missing] 컬럼별 결측 개수")
print(df.isna().sum())


[Missing] 컬럼별 결측 개수
date     3
menu     0
price    1
qty      1
paid     0
sales    1
dtype: int64


### 어떻게 해야 잘 쓸수 있을까?
1. 이 컬럼은 계산 대상인가?
2. 조건 판단 기준인가?
3. 날짜 기준 분석이 필요한가?
4. 이름/분류 역할인가?
이 질문의 답이 dtype 선택 기준이다.


## 3.3) dtype 확인 / 변환의 기본 원칙

### 1. 기억하자 dtype 확인은 항상 전처리의 출발점이다
전처리를 시작할 때 가장 먼저 해야 할 행동은 단 하나다.
``` python
df.dtypes
```
내가 의미로 생각한 타입과 Pandas가 실제로 인식한 타입은 거의 항상 다르기 때문에 항상 dtype를 해야한다.

### 2. dtype 변환 방식은 딱 두가지 방식이다.
실무에서 dtype 변환은 항상 다음 두 가지 중 하나다.
1. 명시적 변환: astype()
2. 파싱 변환: pd.to_numeric, pd.to_datetime
중요한 건 "문법 차이"가 아닌 성격의 차이다.

### 3. 변환 방식 ① astype()
형식이 이미 맞아야 쓰는 방법

astype()는 이렇게 정의하면 된다.
- 값의 형식이 이미 깔끔하다고 가정하고 dtype만 바꾸는 방법

언제 사용 가능한가 ?
- 결측이 없고
- 문자 섞임이 없고
- 이미 타입이 맞을 때

사용시 주의사항
```python
df["qty"].astype(int)
```
- "3개" 같은 값이 있다
- None이 섞여 있다
- 이럴경우 실패하거나 문제가 될수 있다.

### 4. 변환 방식 ② 파싱 변환
to_numeric, to_datetime

문자열을 의미적으로 해석해 dtype를 바꾸는 방법

언제 쓰는가 ?
- 문자열이 섞여 있을 때
- 결측이 있을 때
- 실무에서 받은 원본 데이터일 때

#### 숫자 파싱
'''python
pd.to_numeric(컬럼, errors="coerce")
'''

숫자로 해석 가능한 값만 숫자로 바꾸고 나머지는 결측치(NaN)로 처리한다

#### 날짜 파싱
''' python
pd.to_datetime(컬럼, errors="coerce")
'''

날짜로 해석 가능한 값만 datetime으로 바꾸고 나머지는 날짜형 결측(NaT)로 처리한다

## 3.4) errors="coerce"의 의미 (초보자 필수)

### 1. dtype 변환에서 항상 발생하는 문제
to_numeric, to_datetime 같은 변환은 항상 실패 가능성을 가진다.
- 왜냐하면 실무 데이터에는 이런 값들이 섞여 있기 때문이다.
    - 숫자처럼 보이지만 숫자가 아닌 값
    - 날짜처럼 보이지만 날짜가 아닌 값
    - 아예 값이 없는 경우
즉, 변환 실패는 예외 상황이 아니라 기본적으로 발생하는 상황이다.

### 2. 변환 실패를 처리하는 세 가지 태도
dtype 파싱 함수에는 errors 옵션이 있다.

이 옵션은 "변환이 실패하면 어떻게 할까?"를 묻고 실행한다.

#### (1) errors="raise"
실패하면 즉시 중단
```python
pd.to_numeric(컬럼, errors="raise")
pd.to_datetime(컬럼, errors="raise")
```
변환에 실패하는 값이 하나라도 있으면 에러를 발생시키고 작업을 중단한다
- 데이터 검증 단계에는 유용
- 실무 전처리에는 거의 쓰이지 않음

#### (2) errors="coerce"
실패하면 결측치로 바꾸고 계속 진행
```python
pd.to_numeric(컬럼, errors="coerce")
pd.to_datetime(컬럼, errors="coerce")
```
- 변환이 안 되는 값은 문제가 있는 데이터로 표시하고 일단 분석 흐름을 유지한다

### 3. coerce의 본질을 문장으로 정리하면
errors="coerce"는 "문제 데이터를 숨기는 옵션"이 아니라 "문제 데이터를 눈에 보이게 만드는 전략"이다.

- 숫자 변환 실패 → NaN
- 날짜 변환 실패 → NaT

즉, "이 값은 분석에 쓸 수 없다"는 표시를 데이터에 명확히 남기는 것

그렇기에
- 문제 값을 결측으로 표시하고
- 다음 단계에서 의도적으로 처리한다

# 4) 결측치(Missing Value) 이론

## 4.1) 결측치란 무엇인가?
결측치란 값이 비어 있거나, 알 수 없는 상태를 말한다.

Pandas에서는 보통 이렇게 표시된다.
- 숫자형 결측 → NaN
- 날짜형 결측 → NaT

기억하자
- 결측치는 “이상한 데이터”가 아니라 현실 데이터를 그대로 반영한 상태다.

## 4.2) 실무에서 결측치는 왜 생기는가?

1. 입력 누락
    - 사람이 값을 입력하지 않음
2. 시스템 / 센서 오류
    - 수집 과정에서 값이 빠짐
3. 조인(merge) 실패
    - 다른 테이블과 매칭되지 않음
4. 원래 존재하지 않는 값
    - 옵션 미선택, 해당 항목 없음

결측치는 실수라기보다 데이터 생성 과정의 결과다.

## 4.3) 결측치 처리의 기본 흐름
결측치 처리는 아래 순서를 절대 건너뛰면 안 된다.

1. 결측치가 있는지 확인한다
2. 왜 생겼을지 추측한다
3. 분석 목적에 맞게 처리 전략을 고른다


## 4.4) 결측치 처리 전략
1. 삭제 (dropna)
2. 대체 (fillna)
3. 추정

### ① 삭제 (dropna)
기억하자, 되도록 사용 자제해야 한다.

언제 쓰는가?
- 결측 행이 매우 적을 때
- 결측이 분석 결과를 심각하게 왜곡할 때

특징
- 장점: 단순, 해석이 깔끔
- 단점: 데이터가 줄어들어 분석력이 떨어질 수 있음

```python
df_drop = df.dropna()

```
삭제는 가장 쉬운 방법이지만 가장 위험한 선택일 수 있다.

### ② 대체 (fillna)
결측치를 채운다는 건 값을 추측해서 넣는 행위다.

언제 쓰는가
- 데이터 양을 유지하고 싶을 때
- "비어 있음"이 어느 정도 의미를 가질 때

```python
df_fill = df.copy()
```

### ③ 추정 (개념만 이해하고 넘어가기)
추정은 가장 정교하지만 가장 조심해야 할 방법이다.
- 같은 메뉴의 평균 가격
- 같은 날짜의 평균 매출
- 모델 기반 예측


# 5) 전처리 작업의 대표 패턴
실무에서 전처리를 처음부터 매번 새로 생각하지 않고 자주 반복되는 패턴으로 처리할 수 있게 만드는 실력을 키워보자

## 5.1) 문자열 정리 → 숫자 변환 패턴
이 패턴은 실무에서 가장 많이 쓰이는 전처리 패턴

숫자 계산이 목적인데 값이 문자열이라면 이 패턴을 적용한다.

가격, 수량, 금액처럼 "숫자여야 하는데 문자열로 들어온 컬럼"은 거의 항상 이 과정을 거친다.

언제 쓰나?
- "4,500원"
- " 5000 "
- "3개"
- "무료", "two"

In [246]:
# =========================
# 0) 실무형 더러운 데이터(문자/기호/공백/이상값/결측 섞임)
# =========================
df = pd.DataFrame({
    "menu": ["Americano", "Latte", "Mocha", "Latte", "Americano"],
    "price_raw": ["4,500원", " 5000 ", "무료", None, "5,2oo원"],   # 무료, None, 5,2oo(오타) 포함
    "qty_raw":   ["2", " 1 ", "3개", None, "two"],               # 3개, None, two 포함
})

print("=== 원본 데이터 ===")
print(df)

print("\n=== 원본 데이터 dtypes===")
print(df.dtypes)


=== 원본 데이터 ===
        menu price_raw qty_raw
0  Americano    4,500원       2
1      Latte     5000       1 
2      Mocha        무료      3개
3      Latte      None    None
4  Americano    5,2oo원     two

=== 원본 데이터 dtypes===
menu         object
price_raw    object
qty_raw      object
dtype: object


### 문자열 → 숫자 변환의 표준 순서
이 순서는 공식처럼 기억해도 된다.
1. 불필요한 문자 제거
2. 숫자로 변환
3. 실패 값은 결측으로 표시
4. 결측 처리 전략 선택

#### 1단계 불필요 문자 제거

In [247]:
price_clean = (
    df["price_raw"].astype("string")
    .str.strip()                        # 앞뒤 공백 제거
    .str.replace(",", "", regex=False)  # 쉼표 제거
    .str.replace("원", "", regex=False) # 단위 제거
)

qty_clean = (
    df["qty_raw"].astype("string")
    .str.strip()
    .str.replace("개", "", regex=False) # '3개' → '3'
)
print("=== price_clean ===")
print(price_clean)

print("\n=== qty_clean ===")
print(qty_clean)

=== price_clean ===
0    4500
1    5000
2      무료
3    <NA>
4    52oo
Name: price_raw, dtype: string

=== qty_clean ===
0       2
1       1
2       3
3    <NA>
4     two
Name: qty_raw, dtype: string


#### 2단계 숫자로 변환
- 숫자로 해석 가능한 값 → 숫자
- "무료", "two", 오타 값 → NaN

#### 3단계 실패 값은 결측으로 남긴다
errors="coerce" 덕분에 결측값은 에러가 아니라 결측치(NaN) 로 남는다.

In [248]:
df["price"] = pd.to_numeric(price_clean, errors="coerce")
df["qty"]   = pd.to_numeric(qty_clean,   errors="coerce")
print("=== df[price] ===")
print(df["price"])

print("\n=== df[qty] ===")
print(df["qty"])

=== df[price] ===
0    4500
1    5000
2    <NA>
3    <NA>
4    <NA>
Name: price, dtype: Int64

=== df[qty] ===
0       2
1       1
2       3
3    <NA>
4    <NA>
Name: qty, dtype: Int64


#### 4단계 결측 처리 전략 선택
주의사항
- 연습에서는 0으로 채웠지만 실무에서는 "0이 의미 있는 값인가?"를 반드시 질문해야 한다.

In [249]:
df_fill = df.copy()
df_fill["price"] = df_fill["price"].fillna(0)
df_fill["qty"]   = df_fill["qty"].fillna(0)

df_fill

,menu,price_raw,qty_raw,price,qty
0,Americano,"4,500원",2,4500,2
1,Latte,5000,1,5000,1
2,Mocha,무료,3개,0,3
3,Latte,None,None,0,0
4,Americano,"5,2oo원",two,0,0


In [250]:
# 파생 변수 선언
df_fill["sales"] = df_fill["price"] * df_fill["qty"]

df_fill

,menu,price_raw,qty_raw,price,qty,sales
0,Americano,"4,500원",2,4500,2,9000
1,Latte,5000,1,5000,1,5000
2,Mocha,무료,3개,0,3,0
3,Latte,None,None,0,0,0
4,Americano,"5,2oo원",two,0,0,0


#### (선택) 결측 제거 패턴
```python
df_drop = df.dropna(subset=["price", "qty"]).copy()
df_drop["sales"] = df_drop["price"] * df_drop["qty"]
```
이 선택은 이런 경우에만 적절하게 사용된다.
- 결측이 극히 적을 때
- 결측이 결과를 심각하게 왜곡할 때

## 5.2) 범주값 통일 패턴 (TRUE / True / false …)
이 패턴은 의미는 같지만 표현만 다를 때 사용한다.

값의 의미는 같은데 표기만 다르다면 범주값 통일이 필요하다.

언제 쓰나?
- TRUE / True / true
- FALSE / False / false
- Y / N
- 1 / 0

아무리 같은 의미를 지녔더라도 범주형 데이터는 표기가 다르면 완전히 다른 값으로 인식된다.

### 문자열 → 숫자 변환의 표준 순서
- 문자열 기준으로 통일한다
- 불필요한 공백 / 대소문자 차이를 제거한다
- 의미 기준으로 하나의 값으로 매핑한다
- 매핑 실패 값은 따로 확인한다

In [251]:
# =========================
# 0) 실무에서 흔한 "표기 흔들림" 예시 데이터
# =========================
df = pd.DataFrame({
    "paid": ["TRUE", "True", "true", "FALSE", "False", "false", True, False, None, "  true  ", "Y", "N", "1", "0"],
    "menu": ["A","A","B","B","C","C","D","D","E","E","F","F","G","G"]
})

print("=== 원본 데이터 ===")
print(df)

print("\n=== 원본 데이터 dtypes===")
print(df.dtypes)


=== 원본 데이터 ===
        paid menu
0       TRUE    A
1       True    A
2       true    B
3      FALSE    B
4      False    C
5      false    C
6       True    D
7      False    D
8       None    E
9     true      E
10         Y    F
11         N    F
12         1    G
13         0    G

=== 원본 데이터 dtypes===
paid    object
menu    object
dtype: object


#### 1단계 문자열 기준으로 정규화
비교와 매핑이 안정적으로 가능하도록 형태를 먼저 맞춘다.

In [252]:
paid_norm = (
    df["paid"]
    .astype("string")   # 문자열 기준으로 통일
    .str.strip()        # 앞뒤 공백 제거
    .str.upper()        # 대소문자 통일
)

paid_norm

0      TRUE
1      TRUE
2      TRUE
3     FALSE
4     FALSE
5     FALSE
6      TRUE
7     FALSE
8      <NA>
9      TRUE
10        Y
11        N
12        1
13        0
Name: paid, dtype: string

#### 2단계 의미 기준 매핑 규칙 만들기
매핑 기준은 표기 기준이 아니라 의미 기준이다.

In [253]:
paid_map = {
    "TRUE": True,  "FALSE": False,
    "Y": True,     "N": False,
    "YES": True,   "NO": False,
    "1": True,     "0": False,
    "T": True,     "F": False,
}

paid_map

{'TRUE': True,
 'FALSE': False,
 'Y': True,
 'N': False,
 'YES': True,
 'NO': False,
 '1': True,
 '0': False,
 'T': True,
 'F': False}

#### 3단계 매핑 적용
- 해석할 수 없는 값은 억지로 바꾸지 않고 일단 결측으로 남긴다.

In [254]:
df["paid_bool"] = paid_norm.map(paid_map)
df["paid_bool"] 

0      True
1      True
2      True
3     False
4     False
5     False
6      True
7     False
8       NaN
9      True
10     True
11    False
12     True
13    False
Name: paid_bool, dtype: object

#### 4단계 매핑 실패 값 확인
범주값 통일은 "덮어버리는 작업"이 아닌 문제 데이터를 드러내는 작업이다.

In [255]:
unknown = df[df["paid_bool"].isna()][["paid"]]
unknown


,paid
8,None


#### 5단계 출력

In [256]:
print("\n=== 통일 후 paid_bool ===")
print(df[["paid", "paid_bool"]])


=== 통일 후 paid_bool ===
        paid paid_bool
0       TRUE      True
1       True      True
2       true      True
3      FALSE     False
4      False     False
5      false     False
6       True      True
7      False     False
8       None       NaN
9     true        True
10         Y      True
11         N     False
12         1      True
13         0     False


## 5.3) 파생 피처 생성의 의미

파생 피처는 전처리에서 가장 분석적인 작업이라 할수 있다.

### 파생 피처란 무엇인가?
원본 데이터에는 없지만, 분석 목적을 위해 새로 만들어낸 컬럼이다.

### 필요한 이유?
원본 데이터는 보통 이런 특징을 가진다.
- 저장과 기록 목적
- 분석에 바로 쓰기엔 정보가 흩어져 있음

때문에 다음과 같은 문제가 발생한다.
- “데이터는 있는데 내가 알고 싶은 값은 없다.”

이러한 경우에 파생 피처를 사용하게 된다.

### 파생 피처 생성의 기본 전제
파생 피처를 만들기 전에는 반드시 이 조건이 만족되어야 한다.
- 기존 피처들이 분석 가능한 dtype 상태여야 한다.

때문에 파생 피처의 생성은 항상 후순위 이다.


### 코드로 보기

In [257]:
df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "price": "4,500원", "qty": "2"},
    {"date": "2026/01/02", "menu": "Latte",     "price": "5000",   "qty": 1},
    {"date": "2026-01-03", "menu": "Mocha",     "price": None,     "qty": 3},
    {"date": "invalid",    "menu": "Latte",     "price": "5,000원","qty": None},
])

print("=== 원본 데이터 ===")
print(df)

print("\n=== 원본 데이터 dtypes===")
print(df.dtypes)


=== 원본 데이터 ===
         date       menu   price   qty
0  2026-01-01  Americano  4,500원     2
1  2026/01/02      Latte    5000     1
2  2026-01-03      Mocha    None     3
3     invalid      Latte  5,000원  None

=== 원본 데이터 dtypes===
date     object
menu     object
price    object
qty      object
dtype: object


#### 1단계 기존 피처 전처리 (전제 조건)
- 파생 피처를 만들기 위한 재료를 준비하는 단계

In [258]:
df["price"] = (
    df["price"].astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["qty"]   = pd.to_numeric(df["qty"], errors="coerce")
df["date"]  = pd.to_datetime(df["date"], errors="coerce")


df


,date,menu,price,qty
0,2026-01-01,Americano,4500,2.0
1,NaT,Latte,5000,1.0
2,2026-01-03,Mocha,<NA>,3.0
3,NaT,Latte,5000,NaN


#### 2단계 파생 피처 예시 ①
- 원본에는 sales라는 컬럼이 없었다
- 하지만 분석에서 가장 중요한 값이다

파생 피처는 "없던 값을 만드는 것"이 아닌 흩어진 정보를 하나로 모으는 것이다.

In [259]:
df["sales"] = df["price"] * df["qty"]


df

,date,menu,price,qty,sales
0,2026-01-01,Americano,4500,2.0,9000.0
1,NaT,Latte,5000,1.0,5000.0
2,2026-01-03,Mocha,<NA>,3.0,<NA>
3,NaT,Latte,5000,NaN,<NA>


#### 3단계 파생 피처 예시 ②
- 날짜 자체보다 요일이 분석에 더 중요한 경우가 많다.

In [260]:
df["weekday_en"] = df["date"].dt.day_name()


df

,date,menu,price,qty,sales,weekday_en
0,2026-01-01,Americano,4500,2.0,9000.0,Thursday
1,NaT,Latte,5000,1.0,5000.0,NaN
2,2026-01-03,Mocha,<NA>,3.0,<NA>,Saturday
3,NaT,Latte,5000,NaN,<NA>,NaN


In [261]:
# 한국어 요일이 필요하면 매핑을 추가한다.
weekday_map = {
    "Monday": "월", "Tuesday": "화", "Wednesday": "수",
    "Thursday": "목", "Friday": "금", "Saturday": "토", "Sunday": "일"
}
df["weekday"] = df["weekday_en"].map(weekday_map)

df

,date,menu,price,qty,sales,weekday_en,weekday
0,2026-01-01,Americano,4500,2.0,9000.0,Thursday,목
1,NaT,Latte,5000,1.0,5000.0,NaN,NaN
2,2026-01-03,Mocha,<NA>,3.0,<NA>,Saturday,토
3,NaT,Latte,5000,NaN,<NA>,NaN,NaN


# 6. 파일 입출력 (CSV / Excel / JSON)
"분석의 시작과 끝"

## 6.1) 왜 파일 입출력이 중요한가?
분석에서 파일 입출력은 단순한 저장 기능이 아니다.
- 전처리 결과를 보존한다
- 다른 사람과 공유한다
- 다음 분석 단계의 출발점이 된다

파일 저장은 분석의 부수 작업이 아닌 분석 과정의 일부다.



## 6.2) CSV / Excel / JSON은 언제 쓰는가?

### CSV
분석 결과를 전달한다 → CSV가 기본 선택이다.
- 가장 단순한 표 형태
- 거의 모든 도구에서 읽을 수 있음
- 분석 결과 공유용으로 가장 많이 사용

### Excel (xlsx)
"사람이 직접 본다" → Excel이 편하다.
- 사람이 직접 열어보는 용도
- 보고용, 검토용 데이터에 적합
- 여러 시트를 사용할 수 있음

### JSON
"시스템 간 데이터 교환"→ JSON을 쓴다.
- 구조화된 데이터 형식
- API, 서비스 연동, 로그 저장에 자주 사용
- 테이블보다는 데이터 구조 전달에 강함

### 6.3) CSV 저장 / 읽기

In [262]:
csv_path = "sales.csv"

df.to_csv(csv_path, index=False, encoding="utf-8-sig")
df_csv = pd.read_csv(csv_path)

# index=False
# 인덱스가 의미 있는 데이터가 아니라면 파일에 저장하지 않는다.

# encoding="utf-8-sig"
# 엑셀에서 한글이 깨지는 문제를 줄이기 위해서 변경

### 6.4) Excel 저장 / 읽기

In [263]:
excel_path = "sales.xlsx"

df.to_excel(excel_path, index=False)
df_xlsx = pd.read_excel(excel_path)

### 6.5) JSON 저장 / 읽기

In [264]:
json_path = "sales.json"

df.to_json(
    json_path,
    orient="records",
    force_ascii=False,
    indent=2
)
df_json = pd.read_json(json_path)


#### orient="records"?

이 옵션은 JSON을 다음 형태로 만든다.

```json
[
  {"date": "2026-01-01", "menu": "아메리카노", "sales": 9000},
  {"date": "2026-01-02", "menu": "라떼", "sales": 5000}
]
```